# Train a simple convolutionalNetworks

network structure:
conv-conv-pool-dense-dense-softmax

In [ ]:
import _pickle as pickle
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Helper functions to load raw data
def load_data(data_dir, fname):
    '''
    Transform data features and lables to numpy array.
    X is feature array
    Y is label array
    '''
    with open(os.path.join(data_dir, fname), 'rb') as f:
    
        data = pickle.load(f)
    
    num_samples = len(data)
    X_dim = data[0]['features'].shape
    Y_dim = data[0]['label'].shape
    
    X = np.zeros((num_samples, *X_dim))
    Y = np.zeros((num_samples, *Y_dim))
    
    count = 0
    for d in data:
        X[count] = d['features']
        Y[count] = d['label']
        count += 1
    
    del data
    
    return X, Y   

In [ ]:
# Load raw data 
data_dir = ".\\data"
X_train, Y_train = load_data(data_dir, "training_data.pickle")
X_test, Y_test = load_data(data_dir, "test_data.pickle")

In [ ]:
print (X_train.shape, Y_train.shape)

In [ ]:
print (X_test.shape, Y_test.shape)

In [ ]:
import keras
from keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, MaxPooling1D, Conv2D
from keras.models import Model, Sequential
from keras.optimizers import SGD

In [ ]:
# check if tensorflow is running on GPU
from  tensorflow.python.client  import device_lib
print(device_lib.list_local_devices())

In [ ]:
input_shape = X_train[0].shape
num_classes = Y_train[0].shape[0]
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 padding='same',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
batch_size = 64
epochs = 8

# learning_rate = 0.1
# decay_rate = learning_rate / epochs
# momentum = 0.8

# sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, Y_test))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=64,
                    epochs=8,
                    verbose=1,
                    validation_data=(X_test, Y_test))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
# serialize weights to HDF5
model.save_weights("model.h5")